In [1]:
import pandas as pd
import numpy as np
import re


In [2]:
train_cc = pd.read_csv("F:\GIT_HUB_UPLOAd\_____python_ML_pro\PROJECT_1\data_set/Consumer_Complaints_train.csv")
test_cc = pd.read_csv("F:\GIT_HUB_UPLOAd\_____python_ML_pro\PROJECT_1\data_set/Consumer_Complaints_test_share.csv")

In [3]:
train_cc.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2014-05-15,Credit card,NaN,Billing statement,NaN,NaN,NaN,Wells Fargo & Company,MI,48342,Older American,NaN,Web,2014-05-16,Closed with explanation,Yes,No,856103
1,2014-09-18,Bank account or service,(CD) Certificate of deposit,"Making/receiving payments, sending money",NaN,NaN,NaN,Santander Bank US,PA,18042,NaN,NaN,Referral,2014-09-24,Closed,Yes,No,1034666
2,2014-03-13,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,Equifax,CA,92427,NaN,NaN,Referral,2014-04-03,Closed with non-monetary relief,Yes,No,756363
3,2015-07-17,Credit card,NaN,Billing statement,NaN,"My credit card statement from US Bank, XXXX. X...",Company chooses not to provide a public response,U.S. Bancorp,GA,305XX,Older American,Consent provided,Web,2015-07-17,Closed with monetary relief,Yes,No,1474177
4,2014-11-20,Credit card,NaN,Transaction issue,NaN,NaN,NaN,Bank of America,MA,02127,NaN,NaN,Web,2014-11-28,Closed with explanation,Yes,No,1132572


In [4]:
train_cc.shape,test_cc.shape

((478421, 18), (119606, 17))

In [5]:
train_cc['data'] = 'train'
test_cc['data'] = 'test'
test_cc['Consumer disputed?'] = np.nan



In [6]:
cc_all = pd.concat([train_cc, test_cc], axis=0)


C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [7]:
cc_all.head()

,Company,Company public response,Company response to consumer,Complaint ID,Consumer complaint narrative,Consumer consent provided?,Consumer disputed?,Date received,Date sent to company,Issue,Product,State,Sub-issue,Sub-product,Submitted via,Tags,Timely response?,ZIP code,data
0,Wells Fargo & Company,NaN,Closed with explanation,856103,NaN,NaN,No,2014-05-15,2014-05-16,Billing statement,Credit card,MI,NaN,NaN,Web,Older American,Yes,48342,train
1,Santander Bank US,NaN,Closed,1034666,NaN,NaN,No,2014-09-18,2014-09-24,"Making/receiving payments, sending money",Bank account or service,PA,NaN,(CD) Certificate of deposit,Referral,NaN,Yes,18042,train
2,Equifax,NaN,Closed with non-monetary relief,756363,NaN,NaN,No,2014-03-13,2014-04-03,Incorrect information on credit report,Credit reporting,CA,Account status,NaN,Referral,NaN,Yes,92427,train
3,U.S. Bancorp,Company chooses not to provide a public response,Closed with monetary relief,1474177,"My credit card statement from US Bank, XXXX. X...",Consent provided,No,2015-07-17,2015-07-17,Billing statement,Credit card,GA,NaN,NaN,Web,Older American,Yes,305XX,train
4,Bank of America,NaN,Closed with explanation,1132572,NaN,NaN,No,2014-11-20,2014-11-28,Transaction issue,Credit card,MA,NaN,NaN,Web,NaN,Yes,02127,train


In [8]:
list(zip(cc_all.columns,cc_all.dtypes,cc_all.nunique()))

[('Company', dtype('O'), 3437),
 ('Company public response', dtype('O'), 10),
 ('Company response to consumer', dtype('O'), 7),
 ('Complaint ID', dtype('int64'), 598027),
 ('Consumer complaint narrative', dtype('O'), 92118),
 ('Consumer consent provided?', dtype('O'), 4),
 ('Consumer disputed?', dtype('O'), 2),
 ('Date received', dtype('O'), 1759),
 ('Date sent to company', dtype('O'), 1708),
 ('Issue', dtype('O'), 95),
 ('Product', dtype('O'), 12),
 ('State', dtype('O'), 62),
 ('Sub-issue', dtype('O'), 68),
 ('Sub-product', dtype('O'), 47),
 ('Submitted via', dtype('O'), 6),
 ('Tags', dtype('O'), 3),
 ('Timely response?', dtype('O'), 2),
 ('ZIP code', dtype('O'), 27342),
 ('data', dtype('O'), 2)]

In [9]:
garbage_cols=cc_all.isnull().sum()[cc_all.isnull().sum()>(0.5*cc_all.shape[0])].keys() 

In [10]:
cc_all.drop(columns=garbage_cols,axis=1,inplace=True)

In [11]:
cc_all.drop(columns=["Complaint ID"],inplace=True)

In [12]:
cc_all["Sub-product"]=np.where(cc_all["Sub-product"].isnull(),"not_available",cc_all["Sub-product"])

In [13]:
cc_all["State"]=np.where(cc_all["State"].isnull(),"not_available",cc_all["State"])

In [14]:
cc_all["ZIP code"]=np.where(cc_all["ZIP code"].isnull(),"not_available",cc_all["ZIP code"])

In [15]:
cc_all["Submitted via"]=np.where(cc_all["Submitted via"].isnull(),cc_all["Submitted via"].mode()[0],cc_all["Submitted via"])

In [16]:
import datetime
from datetime import date

In [17]:
d=pd.to_datetime(cc_all["Date received"])
dd=pd.to_datetime(cc_all["Date sent to company"])
cc_all["resolution_gap_days"]=(dd-d)/np.timedelta64(1,'D')
cc_all.drop(columns =["Date received","Date sent to company"], inplace = True)

In [18]:
def dummies(data,var,freq_cutoff=0):
    t=data[var].value_counts(normalize=True)  #get value counts of all the unique data in specified feature variable 
    t=t[t.values>freq_cutoff] #filter the values having count less than specified cutoff
    t=t.sort_values() #sort according to the value counts
    t_min=t.idxmin() # get the data having minimum count 
    t=t.drop([t_min]) # drop that data as we make n-1 dummies 
    categories=t.index # making dummies for rest unique data values

    for cat in categories :
        name=var+'_'+cat
        name=re.sub(" ","",name) 
        name=re.sub("-","_",name)
        name=re.sub("\\?","Q",name) 
        name=re.sub("<","LT_",name) 
        name=re.sub("\\+","",name) 
        name=re.sub("\\/","_",name) 
        name=re.sub(">","GT_",name) 
        name=re.sub("=","EQ_",name)
        name=re.sub(",","",name)
        data[name]=(data[var]==cat)+0 
               
    data=data.drop(columns=[var]) #dropping original feature variable after making its dummies
    return data

In [19]:

cc_all=dummies(cc_all,"Company",0.02)
cc_all=dummies(cc_all,"State",0.04)
cc_all=dummies(cc_all,"ZIP code",0.002)
cc_all=dummies(cc_all,"Submitted via",0.05)
cc_all=dummies(cc_all,"Company response to consumer",0.05)
cc_all=dummies(cc_all,"Timely response?")
cc_all=dummies(cc_all,"Product",0.03)
cc_all=dummies(cc_all,"Sub-product",0.05)
cc_all=dummies(cc_all,"Issue",0.02)


In [21]:
cc_train=cc_all[cc_all['data']=='train'] 
del cc_train['data'] 
cc_test=cc_all[cc_all['data']=='test']
cc_test.drop(['Consumer disputed?','data'],axis=1,inplace=True) 

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [39]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score

In [24]:
train1, train2 = train_test_split(cc_train, test_size = 0.2,random_state=2)

x_train1=train1.drop(["Consumer disputed?"],1)
y_train1=train1["Consumer disputed?"]

x_train2=train2.drop(["Consumer disputed?"],1)
y_train2=train2["Consumer disputed?"]

x_train1.reset_index(drop=True,inplace=True)
y_train1.reset_index(drop=True,inplace=True)

In [28]:
params={'n_estimators':[100,200,500,700],
       'criterion':['gini','entropy'],
       'min_samples_split':[5,7,9],
       'bootstrap':[True],
        'n_jobs':[-1],
       'max_depth':[5,10,20],
       'max_features':[5,10,20,30,40], 
       'min_samples_leaf':[6,8,10]}

## TIME THIS MAX 5 HRS  & ALSO DEPEND YOUR SYSTEM


#________________________________________________________________


#param_dist = {"n_estimators":[100,200,300,500,700,1000],
#"max_features": [5,10,20,25,30,35],
#"bootstrap": [True, False],
#"class_weight":[None,'balanced'],
#"criterion":['entropy','gini'],
#"max_depth":[None,5,10,15,20,30,50,70],
#"min_samples_leaf":[1,2,5,10,15,20],
#"min_samples_split":[2,5,10,15,20]}


## APPALY THIS PARA AUC WILL INCREASE 0.60 OR MORE BUT TAKE MORE TIME MAX 12 HRS


In [25]:
4*2*3*3*5*3

1080

In [26]:
clf = RandomForestClassifier(class_weight='balanced')

In [29]:
random_search = RandomizedSearchCV(clf, param_distributions=params,
                                   n_iter=10,verbose=20)

In [30]:
random_search.fit(x_train1, y_train1)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] n_jobs=-1, n_estimators=500, min_samples_split=9, min_samples_leaf=10, max_features=40, max_depth=20, criterion=entropy, bootstrap=True 
[CV]  n_jobs=-1, n_estimators=500, min_samples_split=9, min_samples_leaf=10, max_features=40, max_depth=20, criterion=entropy, bootstrap=True, score=0.550, total= 7.4min
[CV] n_jobs=-1, n_estimators=500, min_samples_split=9, min_samples_leaf=10, max_features=40, max_depth=20, criterion=entropy, bootstrap=True 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  7.4min remaining:    0.0s


[CV]  n_jobs=-1, n_estimators=500, min_samples_split=9, min_samples_leaf=10, max_features=40, max_depth=20, criterion=entropy, bootstrap=True, score=0.548, total= 7.1min
[CV] n_jobs=-1, n_estimators=500, min_samples_split=9, min_samples_leaf=10, max_features=40, max_depth=20, criterion=entropy, bootstrap=True 


...

[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed: 152.0min remaining:    0.0s


[CV]  n_jobs=-1, n_estimators=100, min_samples_split=9, min_samples_leaf=6, max_features=10, max_depth=10, criterion=gini, bootstrap=True, score=0.557, total=  25.1s
[CV] n_jobs=-1, n_estimators=100, min_samples_split=9, min_samples_leaf=10, max_features=30, max_depth=20, criterion=gini, bootstrap=True 
[CV]  n_jobs=-1, n_estimators=100, min_samples_split=9, min_samples_leaf=10, max_features=30, max_depth=20, criterion=gini, bootstrap=True, score=0.548, total= 1.2min
[CV] n_jobs=-1, n_estimators=100, min_samples_split=9, min_samples_leaf=10, max_features=30, max_depth=20, criterion=gini, bootstrap=True 
[CV]  n_jobs=-1, n_estimators=100, min_samples_split=9, min_samples_leaf=10, max_features=30, max_depth=20, criterion=gini, bootstrap=True, score=0.548, total= 1.2min
[CV] n_jobs=-1, n_estimators=100, min_samples_split=9, min_samples_leaf=10, max_features=30, max_depth=20, criterion=gini, bootstrap=True 
[CV]  n_jobs=-1, n_estimators=100, min_samples_split=9, min_samples_leaf=10, max_fe

[CV]  n_jobs=-1, n_estimators=200, min_samples_split=9, min_samples_leaf=6, max_features=20, max_depth=20, criterion=entropy, bootstrap=True, score=0.549, total= 1.8min
[CV] n_jobs=-1, n_estimators=200, min_samples_split=9, min_samples_leaf=6, max_features=20, max_depth=20, criterion=entropy, bootstrap=True 
[CV]  n_jobs=-1, n_estimators=200, min_samples_split=9, min_samples_leaf=6, max_features=20, max_depth=20, criterion=entropy, bootstrap=True, score=0.553, total= 1.8min
[CV] n_jobs=-1, n_estimators=200, min_samples_split=9, min_samples_leaf=6, max_features=20, max_depth=20, criterion=entropy, bootstrap=True 
[CV]  n_jobs=-1, n_estimators=200, min_samples_split=9, min_samples_leaf=6, max_features=20, max_depth=20, criterion=entropy, bootstrap=True, score=0.555, total= 1.8min
[CV] n_jobs=-1, n_estimators=200, min_samples_split=9, min_samples_leaf=6, max_features=20, max_depth=20, criterion=entropy, bootstrap=True 
[CV]  n_jobs=-1, n_estimators=200, min_samples_split=9, min_samples_le

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed: 204.1min finished


RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight='balanced',
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
      

In [31]:
rf=random_search.best_estimator_
rf

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='entropy', max_depth=5, max_features=30,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [32]:
rf.fit(x_train1,y_train1)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='entropy', max_depth=5, max_features=30,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [33]:
rf.classes_

array(['No', 'Yes'], dtype=object)

In [34]:
train1_score=rf.predict_proba(x_train1)[:,1]
train1_score

array([0.57371114, 0.42906499, 0.53383075, ..., 0.49012505, 0.38866634,
       0.48867183])

In [35]:
train2_score=rf.predict_proba(x_train2)[:,1]
train2_score

array([0.34066152, 0.39354364, 0.56713979, ..., 0.58178323, 0.48797836,
       0.57602425])

In [36]:
train1_classes=(train1_score>0.5).astype(int)
y_train1_classes=np.where(y_train1=="Yes",1,0)
train1_score[train1_score>0.5].size*100/train1_score.size #percent of Yes in train 1

49.40350528824046

In [37]:
train2_classes=(train2_score>0.5).astype(int)
y_train2_classes=np.where(y_train2=="Yes",1,0)
train2_score[train2_score>0.5].size*100/train2_score.size #percent of Yes in train 2

49.15085959136751

In [40]:
roc_auc_score(y_train1, train1_classes) #auc score of train 1

0.5776580089896366

In [41]:
roc_auc_score(y_train2, train2_classes) #auc score of train 2

0.5784939964383135

In [43]:
test_score=rf.predict_proba(cc_test)[:,1]

In [44]:
test_score
test_classes=np.where(test_score>0.5,"Yes","No")

In [45]:
test_classes[test_classes=="No"].size

60674

In [46]:
train1['Consumer disputed?'].value_counts()

No     301424
Yes     81312
Name: Consumer disputed?, dtype: int64

In [ ]:
# params={'n_estimators':[100,200,500,700],
#        'criterion':['gini','entropy'],
#        'min_samples_split':[5,7,9],
#        'bootstrap':[True],
#         'n_jobs':[-1],
#        'max_depth':[5,10,20],
#        'max_features':[5,10,20,30,40], 
#        'min_samples_leaf':[6,8,10]}

## TIME THIS MAX 5 HRS  & ALSO DEPEND YOUR SYSTEM


#________________________________________________________________



#param_dist = {"n_estimators":[100,200,300,500,700,1000],
#"max_features": [5,10,20,25,30,35],
#"bootstrap": [True, False],
#"class_weight":[None,'balanced'],
#"criterion":['entropy','gini'],
#"max_depth":[None,5,10,15,20,30,50,70],
#"min_samples_leaf":[1,2,5,10,15,20],
#"min_samples_split":[2,5,10,15,20]}


## APPALY THIS PARA AUC WILL INCREASE 0.60 OR MORE BUT TAKE MORE TIME MAX 12 HRS
